<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_Architecture_Search_(NAS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import random

# Model definition
class SimpleNASModel(nn.Module):
    def __init__(self, layer_sizes):
        super(SimpleNASModel, self).__init__()
        layers = []
        for in_size, out_size in zip(layer_sizes[:-1], layer_sizes[1:]):
            layers.append(nn.Linear(in_size, out_size))
            layers.append(nn.ReLU())
        self.net = nn.Sequential(*layers[:-1])  # Remove the last ReLU

    def forward(self, x):
        return self.net(x)

# Evaluate model function
def evaluate_model(model, train_loader, val_loader, device, num_epochs=1):
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    loss_fn = nn.CrossEntropyLoss()

    # Train the model
    model.train()
    for epoch in range(num_epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            data = data.view(data.size(0), -1)  # Flatten input
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()

    # Evaluate the model
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            data = data.view(data.size(0), -1)
            output = model(data)
            val_loss += loss_fn(output, target).item()
    return val_loss / len(val_loader)

# Random search function
def random_search(search_space, train_loader, val_loader, device, num_samples):
    best_model = None
    best_loss = float('inf')
    for _ in range(num_samples):
        layer_sizes = random.choice(search_space)
        model = SimpleNASModel(layer_sizes)
        loss = evaluate_model(model, train_loader, val_loader, device)
        print(f"Architecture: {layer_sizes}, Validation Loss: {loss:.4f}")
        if loss < best_loss:
            best_loss = loss
            best_model = model
    return best_model

# Dataset and data loaders
def get_data_loaders(batch_size=64):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    val_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader

# Main script
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_loader, val_loader = get_data_loaders()

    # Define search space
    search_space = [
        [784, 128, 64, 10],
        [784, 256, 128, 10],
        [784, 128, 128, 10],
    ]

    # Perform random search
    print("Performing random search...")
    best_model = random_search(search_space, train_loader, val_loader, device, num_samples=5)
    print("Best model found!")